01. Business Problem and Introduction  
02. Data Loading and Initial Exploration  
03. Data Cleaning and Preprocessing  
04. Feature Engineering  
05. Exploratory Data Analysis (EDA)  
06. Modeling Strategy and Data Splitting  
07. Model Training and Evaluation  
08. Interpretation and Business Recommendations  
09. Conclusion and Next Steps  



## Business Context

Type II diabetes is a chronic condition affecting over 37 million Americans. In 2017 alone, diabetes-related healthcare costs exceeded $327 billion in the U.S., with a significant portion attributed to hospital readmissions. Unplanned readmissions strain the healthcare system and may indicate gaps in treatment or patient management.

### Readmission Classes
- **NO**: No readmission (53.9%)
- **<30**: Readmitted within 30 days (34.9%)
- **>30**: Readmitted after 30 days (11.2%)

### Objective
Predict patient readmission class using historical medical records. Accurate predictions can:
- Reduce avoidable readmissions
- Enable targeted care
- Save costs and improve patient outcomes

### References
- CDC (https://www.cdc.gov/diabetes/data/index.html)
- Kaggle dataset (https://www.kaggle.com/datasets/brandao/diabetes)
- PubMed article (https://pubmed.ncbi.nlm.nih.gov/24804245/)


Citation example: CDC, PubMed, Kaggle links from assignment

# 📁 02. Data Loading and Initial Exploration

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Load dataset
df = pd.read_csv("diabetic_readmission_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'diabetic_readmission_data.csv'

In [ ]:
# Dataset overview
print("Shape:", df.shape)
df.head()

In [ ]:
# Number of entries
len(df)

In [ ]:
# Displaying first 10 rows of data - so we can see every feature
df.head(10).T

In [ ]:
# Display basic summary statistics for all columns
print("Summary Statistics:")
print(df.describe(include='all'))

In [ ]:
# Checking for missing values in dataset
# In the dataset missing values are represented as '?' sign
for col in df.columns:
    if df[col].dtype == object:
         print(col,df[col][df[col] == '?'].count())

In [ ]:
# Summarised version

# Check for missing values represented by '?'
missing_values = df.applymap(lambda x: x == '?').sum()

# Print the count of missing values for each column
print(missing_values[missing_values > 0])

In [ ]:
# Calculate the percentage of missing values for each column
missing_percentage = (missing_values / len(df)) * 100

# Filter for columns with missing values
missing_percentage_filtered = missing_percentage[missing_percentage > 0]

# Convert numerical values to string with a trailing '%' sign and two decimal places
missing_percentage_formatted = missing_percentage_filtered.apply(lambda x: f"{x:.2f}%")

# Print a title
print("===== Percentage of Missing Values per Column =====")

# Print the formatted percentages
missing_percentage_formatted


## Create is_missing column for weight and payer_code

Since weight and payer_code have >30% of rows missing - we will create is_missing column because it could be a meaningful signal
- perhaps patients who did not have a particular measurement recorded are more (or less) likely to be readmitted.
- maybe certain hospitals only record weight for high-risk patients

In [ ]:
# Duplicate df with is_missing column for weight and payer_code

# Create a copy of the DataFrame
df_copy = df.copy()

# Create 'is_missing_weight' and 'is_missing_payer_code' columns
df_copy['is_missing_weight'] = df_copy['weight'] == '?'
df_copy['is_missing_payer_code'] = df_copy['payer_code'] == '?'
df_copy

In [ ]:
# Gender was coded differently (either Male, Female, or Unknonw/Invalid) so we use a custom count for this one
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count())

In [ ]:
# Column data types
df.dtypes.value_counts()

In [ ]:
# Identify numerical (int64) columns

# Assuming 'df' is your DataFrame (as defined in the provided code)
numerical_cols = df.select_dtypes(include=['int64']).columns
numerical_cols

In [ ]:
# prompt: Identify categorical (object) columns

# Identify categorical (object) columns
categorical_cols = df.select_dtypes(include=['object']).columns
categorical_cols


## Initial Exploratory Data Analysis

### Univariate Analysis

#### Histograms

In [ ]:
# Histograms for numeric features to visualize distributions
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols].hist(bins=30, figsize=(15, 10))
plt.suptitle("Histograms of Numeric Features")
plt.show()

#### Boxplots

In [ ]:
# Boxplots for each numeric feature to help identify outliers
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

#### Bar Charts for Categorical Variables:

For features like:
- readmission class and
- diagnosis codes,

**Why?**
<br>
To observe the frequency distribution and identify any significant imbalances.

In [ ]:
# Bar chart for diagnosis codes
plt.figure(figsize=(12, 6))
df['diag_1'].value_counts().nlargest(20).plot(kind='bar') # Plotting top 20 diagnosis codes
plt.title('Top 20 diag_1 Code Frequencies')
plt.xlabel('Diagnosis Code')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.show()


In [ ]:
# Bar chart for diag_2
plt.figure(figsize=(12, 6))
df['diag_2'].value_counts().nlargest(20).plot(kind='bar')
plt.title('Top 20 diag_2 Frequencies')
plt.xlabel('diag_2')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Bar chart for diag_3
plt.figure(figsize=(12, 6))
df['diag_3'].value_counts().nlargest(20).plot(kind='bar')
plt.title('Top 20 diag_3 Frequencies')
plt.xlabel('diag_3')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()

## Multivariate Analysis

In [ ]:
# Correlation heatmaps for numerical_cols to spot multicollinearity.

# Correlation Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(df[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Numerical Features')
plt.show()


Analyze features like age, gender, or admission details, which might provide insights into risk factors for early versus late readmissions.


In [ ]:
# Bar Plot or Count Plot: If age is categorical (like the bracketed format), a bar chart showing the count of patients in each age group helps reveal whether the dataset skews older or younger.

# Assuming 'df' is your DataFrame (as defined in the provided code)
plt.figure(figsize=(8, 6))
sns.countplot(x='age', data=df)
plt.title('Count of Patients in Each Age Group')
plt.xlabel('Age Group')
plt.ylabel('Number of Patients')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()


In [ ]:
# Apply chi-square tests (for categorical age brackets)

from scipy.stats import chi2_contingency

# Assuming 'df' is your DataFrame (as defined in the provided code)
# Create a contingency table
contingency_table = pd.crosstab(df['age'], df['readmitted'])

# Perform the Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

# Interpret the results
alpha = 0.05  # Significance level

if p < alpha:
    print("There is a statistically significant association between age and readmission.")
else:
    print("There is no statistically significant association between age and readmission.")


In [ ]:
# Function to convert an age bracket to its midpoint

def convert_age_bracket_to_midpoint(age_bracket):
    # Remove square brackets if they exist
    age_bracket = age_bracket.strip('[]')
    # Split the string into lower and upper bounds using the hyphen
    parts = age_bracket.split('-')
    # Check if we have two parts and try to compute the midpoint
    if len(parts) == 2:
        lower = parts[0]
        upper = parts[1]

        # Handle potential closing parenthesis in upper bound
        if upper.endswith(')'):
            upper = upper[:-1]  # Remove the ')'

        try:
            lower = float(lower)
            upper = float(upper)
            # Calculate midpoint for half-open interval: (lower + upper - 1)/2
            # Since the upper bound is exclusive, we use upper-1 to make it inclusive.
            midpoint = (lower + upper) / 2
            return midpoint
        except ValueError:
            return None
    return None
# Apply the function to the 'age' column and create a new column 'age_midpoint'
df['age_midpoint'] = df['age'].apply(convert_age_bracket_to_midpoint)

# Display the original age and the computed midpoint for a few records
print(df[['age', 'age_midpoint']].head())

In [ ]:
# Distribution Plot: If you transform the brackets into a numeric variable (e.g., midpoints), you can create a histogram or density plot to see the distribution of ages.

# Create the distribution plot
plt.figure(figsize=(10, 6))
sns.histplot(df['age_midpoint'], kde=True, bins=10)  # Use kde for density plot
plt.title('Distribution of Patient Ages')
plt.xlabel('Age (Midpoint of Bracket)')
plt.ylabel('Frequency/Density')
plt.show()


In [ ]:
# Readmission vs. Age: Perform a group-by operation (e.g., df.groupby('age')['readmission'].value_counts(normalize=True)) to see how readmission rates vary by age group. This might reveal that certain age brackets have higher or lower readmission probabilities.

# Group by age and get normalized readmission counts
readmission_by_age = df.groupby('age')['readmitted'].value_counts(normalize=True)
readmission_by_age


In [ ]:
# Other Features vs. Age: Explore how other variables (e.g., gender, diagnosis codes) distribute across age groups to spot any interesting correlations.

# Assuming 'df' is your DataFrame (as defined in the provided code)

# Group data by age and gender, then count readmissions
readmission_by_age_gender = df.groupby(['age', 'gender'])['readmitted'].value_counts(normalize=True).unstack()

# Plotting readmission rates by age and gender
readmission_by_age_gender.plot(kind='bar', stacked=True, figsize=(15, 8))
plt.title('Readmission Rates by Age and Gender')
plt.xlabel('Age Group')
plt.ylabel('Proportion of Readmissions')
plt.xticks(rotation=45)
plt.legend(title='Readmission Class')
plt.tight_layout()
plt.show()


In [ ]:
# Group data by age and diagnosis code, then count readmissions
# (Example using diag_1, repeat for diag_2 and diag_3)

#The issue with the original plotting was that it tried to plot every combination of age, diag_1 and readmission status as a bar.
#This resulted in an incomprehensible graph due to the sheer number of bars.
# We can instead look at readmission rates for the top N diagnoses in each age group.


def plot_readmissions_by_age_diagnosis(df, diagnosis_col, top_n=10):
    for age_group in df['age'].unique():
        # Filter data for the current age group
        age_df = df[df['age'] == age_group]

        # Get the top N diagnoses for this age group
        top_diagnoses = age_df[diagnosis_col].value_counts().nlargest(top_n).index

        # Filter the data to include only the top diagnoses
        filtered_df = age_df[age_df[diagnosis_col].isin(top_diagnoses)]

        # Group by diagnosis code and calculate readmission rates
        readmission_rates = filtered_df.groupby(diagnosis_col)['readmitted'].value_counts(normalize=True).unstack()


        # Plotting readmission rates
        plt.figure(figsize=(12, 6))  # Adjust figure size as needed
        readmission_rates.plot(kind='bar', stacked=True)
        plt.title(f'Readmission Rates for Top {top_n} {diagnosis_col} in Age Group: {age_group}')
        plt.xlabel(diagnosis_col)
        plt.ylabel('Proportion of Readmissions')
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Readmission Class')
        plt.tight_layout()
        plt.show()

# Call the function for each diagnosis column
plot_readmissions_by_age_diagnosis(df, 'diag_1')
plot_readmissions_by_age_diagnosis(df, 'diag_2')
plot_readmissions_by_age_diagnosis(df, 'diag_3')


In [ ]:
# Aalyze features like age, gender, or admission details, which might provide insights into risk factors for early versus late readmissions.

# Analyze features like age, gender, and admission details for readmission risk.

# Assuming 'df' is your DataFrame (as defined in the previous code)


# 2. Gender:
# Compare readmission rates between genders
gender_readmission = df.groupby('gender')['readmitted'].value_counts(normalize=True).unstack()
print(gender_readmission)

gender_readmission.plot(kind='bar', stacked=True)
plt.title('Readmission Rates by Gender')
plt.show()


In [ ]:
# 3. Admission Details:
# Analyze admission type, admission source, and time in hospital for readmission correlation.

# Admission Type
admission_type_readmission = df.groupby('admission_type_id')['readmitted'].value_counts(normalize=True).unstack()
print(admission_type_readmission)
admission_type_readmission.plot(kind='bar', stacked=True)
plt.title('Readmission Rates by Admission Type')
plt.show()

# Admission Source
admission_source_readmission = df.groupby('admission_source_id')['readmitted'].value_counts(normalize=True).unstack()
print(admission_source_readmission)
admission_source_readmission.plot(kind='bar', stacked=True)
plt.title('Readmission Rates by Admission Source')
plt.show()

# Time in Hospital
plt.figure(figsize=(8,6))
sns.boxplot(x='readmitted',y='time_in_hospital',data=df)
plt.title('Time in Hospital vs Readmission')
plt.show()

#Further analysis can include:
# - Combining features (e.g., age and gender)
# - Statistical tests (e.g., chi-squared test, t-test) to determine statistical significance
# - More detailed visualizations

# 📘 03. Data Cleaning and Preprocessing

In [ ]:
# Flag missing values
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[f"{col}_is_missing"] = df[col].isnull()

# Drop fully null columns
df = df.dropna(axis=1, how='all')

# Convert age ranges to numeric average
df['age'] = df['age'].str.replace('[', '', regex=False).str.replace(')', '', regex=False)
df['age'] = df['age'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2)

# 📘 04. Feature Engineering

In [ ]:
# Simplify ICD-9 codes to numeric prefix
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[col] = df[col].astype(str).str.extract(r'(\d+)', expand=False).astype(float)
    df[col] = df[col].fillna(0)  # 0 = unknown/invalid

In [ ]:
# Encode categorical features
categorical_cols = df.select_dtypes(include='object').columns.drop('readmitted')
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

## Additional from Cam

In [ ]:
# Check if this is_missing_weight and is_missing_payer_code correlates with the readmission outcome. If the correlation or importance is high

# Calculate the correlation between 'is_missing_weight', 'is_missing_payer_code', and 'readmitted'
correlation_weight = df_copy['is_missing_weight'].corr(df_copy['readmitted'] != 'NO')
correlation_payer = df_copy['is_missing_payer_code'].corr(df_copy['readmitted'] != 'NO')

print(f"Correlation between is_missing_weight and readmission: {correlation_weight}")
print(f"Correlation between is_missing_payer_code and readmission: {correlation_payer}")

# Using RandomForestClassifier to assess feature importance
X = df_copy[['is_missing_weight', 'is_missing_payer_code']]  # Features
y = df_copy['readmitted'] != 'NO' # Target variable (readmitted or not)


# Handle non-numeric data (if any) in the features. Assuming boolean values already are represented numerically.
X = X.astype(int) # Convert boolean values to integers (0 or 1) for the classifier

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

print("\nFeature Importances:")
for feature, importance in zip(['is_missing_weight', 'is_missing_payer_code'], feature_importances):
    print(f"{feature}: {importance}")

# 📈 05. Exploratory Data Analysis (EDA)

In [ ]:
# Target distribution
sns.countplot(x='readmitted', data=df)
plt.title("Readmission Distribution")
plt.show()

In [ ]:
# Calculate the distribution of the readmission classes
readmission_counts = df['readmitted'].value_counts()
print("Readmission Counts:")
print(readmission_counts)

# Plot the distribution as a bar chart
plt.figure(figsize=(8, 6))
readmission_counts.plot(kind='bar', color=['skyblue', 'salmon', 'lightgreen'])
plt.title("Distribution of Readmission Classes")
plt.xlabel("Readmission Category")
plt.ylabel("Number of Cases")
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Only include numeric columns for correlation
numeric_df = df_encoded.select_dtypes(include=[np.number])

# Plot correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_df.corr(), cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.show()

# 📘 06. Modeling Strategy and Data Splitting

In [ ]:
X = df_encoded.drop('readmitted', axis=1)
y = df['readmitted']

# Maintain class balance
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, stratify=y_train_val)

# 📘 07. Model Training and Evaluation

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Validation results
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

# 📊 Feature importances
importances = pd.Series(model.feature_importances_, index=X.columns)
importances.nlargest(10).plot(kind='barh')
plt.title("Top 10 Feature Importances")
plt.show()

# 📘 08. Interpretation and Business Recommendations

## Key Insights
- Feature importance analysis shows top predictors: e.g. number of medications, time in hospital.
- The model can identify high-risk readmissions, especially <30 days, which are the most costly.

## Recommended Actions
- Use predictions to flag high-risk patients for follow-up.
- Allocate nurse case managers for <30 day risk patients.
- Educate patients before discharge to reduce recurrence.

## Cost Savings Potential
If model reduces <30 day readmissions by even 5%, estimated savings = millions annually (based on per patient cost of ~$13k).


# 📘 09. Conclusion and Next Steps

## Summary
- We built a predictive model for hospital readmission among diabetic patients.
- Our model improved upon baseline accuracy (53.9%).
- We identified key variables and actionable insights for healthcare providers.

## Limitations
- Many missing values
- Diagnoses not granularly explored (ICD-9 clusters could be used)

## Next Steps
- Experiment with multiclass + binary model combo
- Try advanced models (XGBoost, LightGBM)
- Apply cost-sensitive learning or upsampling minority class
